In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 17:24:54.845210 140065343878976 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 17:24:54.852625 140065343878976 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 17:24:54.858213 140065343878976 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 17:24:54.863711 140065343878976 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 17:24:54.868380 140065343878976 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 17:24:54.876400 140065343878976 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 17:24:54.917336 140065343878976 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 17:24:54.991344 140065343878976 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9261 train_acc= 0.3000 val_loss= 1.9281 val_acc= 0.3500 time= 1.1037
Epoch: 0002 train_loss= 1.9055 train_acc= 0.2929 val_loss= 1.9096 val_acc= 0.3500 time= 0.0172
Epoch: 0003 train_loss= 1.8836 train_acc= 0.2929 val_loss= 1.8904 val_acc= 0.3500 time= 0.0187
Epoch: 0004 train_loss= 1.8608 train_acc= 0.2929 val_loss= 1.8707 val_acc= 0.3500 time= 0.0224
Epoch: 0005 train_loss= 1.8378 train_acc= 0.3000 val_loss= 1.8513 val_acc= 0.3500 time= 0.0187
Epoch: 0006 train_loss= 1.8157 train_acc= 0.3000 val_loss= 1.8329 val_acc= 0.3500 time= 0.0186
Epoch: 0007 train_loss= 1.7948 train_acc= 0.3000 val_loss= 1.8156 val_acc= 0.3500 time= 0.0218
Epoch: 0008 train_loss= 1.7755 train_acc= 0.3000 val_loss= 1.7999 val_acc= 0.3500 time= 0.0202
Epoch: 0009 train_loss= 1.7580 train_acc= 0.3000 val_loss= 1.7860 val_acc= 0.3500 time= 0.0190
Epoch: 0010 train_loss= 1.7419 train_acc= 0.3000 val_loss= 1.7736 val_acc= 0.3500 time= 0.0237
Epoch: 0011 train_loss= 1.7277 train_acc= 0.3000 v

Epoch: 0092 train_loss= 0.7048 train_acc= 0.9286 val_loss= 1.0047 val_acc= 0.7900 time= 0.0245
Epoch: 0093 train_loss= 0.6973 train_acc= 0.9214 val_loss= 0.9977 val_acc= 0.7867 time= 0.0248
Epoch: 0094 train_loss= 0.6905 train_acc= 0.9214 val_loss= 0.9916 val_acc= 0.7800 time= 0.0189
Epoch: 0095 train_loss= 0.6841 train_acc= 0.9214 val_loss= 0.9861 val_acc= 0.7767 time= 0.0192
Epoch: 0096 train_loss= 0.6779 train_acc= 0.9214 val_loss= 0.9804 val_acc= 0.7767 time= 0.0185
Epoch: 0097 train_loss= 0.6712 train_acc= 0.9286 val_loss= 0.9741 val_acc= 0.7767 time= 0.0240
Epoch: 0098 train_loss= 0.6638 train_acc= 0.9286 val_loss= 0.9678 val_acc= 0.7800 time= 0.0181
Epoch: 0099 train_loss= 0.6563 train_acc= 0.9286 val_loss= 0.9619 val_acc= 0.7800 time= 0.0231
Epoch: 0100 train_loss= 0.6497 train_acc= 0.9286 val_loss= 0.9572 val_acc= 0.7767 time= 0.0224
Epoch: 0101 train_loss= 0.6442 train_acc= 0.9357 val_loss= 0.9539 val_acc= 0.7833 time= 0.0246
Epoch: 0102 train_loss= 0.6387 train_acc= 0.9357 v

Epoch: 0181 train_loss= 0.3830 train_acc= 0.9786 val_loss= 0.7631 val_acc= 0.8333 time= 0.0170
Epoch: 0182 train_loss= 0.3842 train_acc= 0.9786 val_loss= 0.7604 val_acc= 0.8400 time= 0.0175
Epoch: 0183 train_loss= 0.3871 train_acc= 0.9714 val_loss= 0.7611 val_acc= 0.8367 time= 0.0205
Epoch: 0184 train_loss= 0.3864 train_acc= 0.9643 val_loss= 0.7607 val_acc= 0.8367 time= 0.0177
Epoch: 0185 train_loss= 0.3818 train_acc= 0.9786 val_loss= 0.7581 val_acc= 0.8433 time= 0.0178
Epoch: 0186 train_loss= 0.3762 train_acc= 0.9786 val_loss= 0.7563 val_acc= 0.8400 time= 0.0231
Epoch: 0187 train_loss= 0.3723 train_acc= 0.9857 val_loss= 0.7560 val_acc= 0.8433 time= 0.0189
Epoch: 0188 train_loss= 0.3706 train_acc= 0.9857 val_loss= 0.7572 val_acc= 0.8333 time= 0.0250
Epoch: 0189 train_loss= 0.3690 train_acc= 0.9857 val_loss= 0.7581 val_acc= 0.8133 time= 0.0303
Epoch: 0190 train_loss= 0.3671 train_acc= 0.9857 val_loss= 0.7583 val_acc= 0.8233 time= 0.0192
Epoch: 0191 train_loss= 0.3654 train_acc= 0.9786 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8010
accuracy = 0.8140


# Gaussian Graph Convolution

In [15]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

W1110 17:25:00.305068 140065343878976 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:28: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 17:25:00.306314 140065343878976 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update

In [16]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
# model.add_loss()

In [17]:
from ggcn.losses import kl_reg

In [18]:
model.add_loss(kl_reg(*H1), H1)

W1110 17:25:00.350755 140065343878976 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/losses.py:27: kl_divergence (from tensorflow.python.ops.distributions.kullback_leibler) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [19]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [20]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9379 train_acc= 0.3214 val_loss= 1.9404 val_acc= 0.2500 time= 0.4586
Epoch: 0002 train_loss= 1.9273 train_acc= 0.4214 val_loss= 1.9341 val_acc= 0.2833 time= 0.0250
Epoch: 0003 train_loss= 1.9165 train_acc= 0.4786 val_loss= 1.9250 val_acc= 0.3600 time= 0.0237
Epoch: 0004 train_loss= 1.9042 train_acc= 0.5071 val_loss= 1.9170 val_acc= 0.3733 time= 0.0195
Epoch: 0005 train_loss= 1.8910 train_acc= 0.5214 val_loss= 1.9063 val_acc= 0.4233 time= 0.0178
Epoch: 0006 train_loss= 1.8762 train_acc= 0.5071 val_loss= 1.8980 val_acc= 0.4267 time= 0.0188
Epoch: 0007 train_loss= 1.8634 train_acc= 0.4857 val_loss= 1.8862 val_acc= 0.4400 time= 0.0182
Epoch: 0008 train_loss= 1.8498 train_acc= 0.5571 val_loss= 1.8758 val_acc= 0.4367 time= 0.0192
Epoch: 0009 train_loss= 1.8308 train_acc= 0.5500 val_loss= 1.8655 val_acc= 0.4300 time= 0.0194
Epoch: 0010 train_loss= 1.8167 train_acc= 0.5429 val_loss= 1.8519 val_acc= 0.4267 time= 0.0183
Epoch: 0011 train_loss= 1.8018 train_acc= 0.5500 v

Epoch: 0092 train_loss= 0.2411 train_acc= 0.9786 val_loss= 0.6485 val_acc= 0.8367 time= 0.0193
Epoch: 0093 train_loss= 0.2347 train_acc= 0.9786 val_loss= 0.6447 val_acc= 0.8333 time= 0.0245
Epoch: 0094 train_loss= 0.2278 train_acc= 0.9786 val_loss= 0.6408 val_acc= 0.8400 time= 0.0185
Epoch: 0095 train_loss= 0.2220 train_acc= 0.9786 val_loss= 0.6357 val_acc= 0.8400 time= 0.0240
Epoch: 0096 train_loss= 0.2133 train_acc= 0.9786 val_loss= 0.6333 val_acc= 0.8400 time= 0.0281
Epoch: 0097 train_loss= 0.2086 train_acc= 0.9786 val_loss= 0.6293 val_acc= 0.8400 time= 0.0282
Epoch: 0098 train_loss= 0.2028 train_acc= 0.9786 val_loss= 0.6263 val_acc= 0.8367 time= 0.0287
Epoch: 0099 train_loss= 0.1965 train_acc= 0.9786 val_loss= 0.6223 val_acc= 0.8367 time= 0.0283
Epoch: 0100 train_loss= 0.1910 train_acc= 0.9786 val_loss= 0.6200 val_acc= 0.8367 time= 0.0285
Epoch: 0101 train_loss= 0.1866 train_acc= 0.9786 val_loss= 0.6169 val_acc= 0.8367 time= 0.0286
Epoch: 0102 train_loss= 0.1817 train_acc= 0.9786 v

In [21]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.5910
accuracy = 0.8100


# Attacks